# Experiments

## Import des fichiers

In [4]:
#Temps et fichiers
import os
import warnings
import time
from datetime import timedelta

#Manipulation de données
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from functools import partial


#Modélisation
from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_tweedie_deviance
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, auc

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV# the keys can be accessed with final_pipeline.get_params().keys()
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

#Text
import re

#Evaluation
from sklearn.metrics import f1_score, confusion_matrix


#Visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px


#Tracking d'expérience
import mlflow
import mlflow.sklearn

## Utilisation du code du projet packagé

In [5]:
#Cette cellule permet d'appeler la version packagée du projet et d'en assurer le reload avant appel des fonctions
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration de l'experiment MLFlow

In [6]:
mlflow.tracking.get_tracking_uri()

'/mnt/experiments'

## Chargement des données

In [7]:
data_folder = os.path.join('/mnt', 'data', 'raw')
all_raw_files = [os.path.join(data_folder, fname)
                    for fname in os.listdir(data_folder)]
all_raw_files

['/mnt/data/raw/Data',
 '/mnt/data/raw/Data.zip',
 '/mnt/data/raw/expo_test.csv',
 '/mnt/data/raw/expo_train.csv',
 '/mnt/data/raw/illustration',
 '/mnt/data/raw/primes2019.csv',
 '/mnt/data/raw/sin_train.csv']

In [8]:
random_state=42

In [42]:
expo_train = pd.read_csv(all_raw_files[3], encoding='utf8', sep=',' )

In [26]:
expo_train.head()

,Unnamed: 0,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,384538,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,0,1,0,0,0.0,0,5,2017
1,441079,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,0,1,0,0,0.0,0,9,2018
2,119668,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,0,1,0,0,0.0,0,11,2017
3,5124,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,0,1,0,0,0.0,0,13,2018
4,130065,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,0,1,0,0,0.0,0,14,2017


In [28]:
expo_test = pd.read_csv(all_raw_files[2], encoding='utf8', sep=',' )
expo_test.head()

,Unnamed: 0,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,141755,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,4.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B43,0,1,0,0,NaN,0,2,2019
1,81642,0.468493,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,0.0,NIVEAU_1,C22,0,1,0,0,0.0,0,10,2019
2,60207,0.852055,ESSENTIEL,PRINCIPALE,APPARTEMENT,0.0,PROPRIO,JUR1,0.0,NIVEAU_1,B32,0,1,0,0,0.0,0,18,2019
3,164160,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,50000.0,NIVEAU_1,C6,0,1,1,0,0.0,0,19,2019
4,33416,0.079452,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,9000.0,NIVEAU_1,C6,0,1,0,0,0.0,0,39,2019


In [32]:
sin_train = pd.read_csv(all_raw_files[6], encoding='utf8', sep=';', decimal=',' )
sin_train.head()

,id,NB,COUT,ANNEE
0,15,1,521.43,2017
1,277,1,3000.19,2016
2,643,1,26.33,2018
3,668,1,462.36,2017
4,730,1,640.88,2018


In [34]:
primes2019 = pd.read_csv(all_raw_files[5], encoding='utf8', sep=';', decimal=',' )
primes2019.head()

,id,prime
0,1000001,81.878323
1,1000002,92.986674
2,1000003,34.635234
3,1000004,70.135502


## EDA

On commence par créer un profile report pour se donner une idée

In [35]:
pr = ProfileReport(expo_train).to_file(output_file='expo_train.html')

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4594: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
pr = ProfileReport(expo_test).to_file(output_file='expo_test.html')

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
pr = ProfileReport(sin_train).to_file(output_file='sin_train.html')

Summarize dataset:   0%|          | 0/17 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Insight

On voit plusieurs pb :
- une première colonne **unnamed** dans les expo_train et expo_test
- expo_train fait état de **sinistres de types 1 et 2**, alors que cette info n'est pas dans sinistres

**Question 1 :** On vérifie si un Id correspond bien à une police x une année d'observation

In [75]:
expo_train[['id', 'ANNEE']].groupby('id').agg('count').describe()

,ANNEE
count,155651.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


**=>** c'est bien le cas

**A FAIRE :** du coup on peut calculer le nombre de sinistres antérieurs

**Question 2 :** est-ce qu'on retrouve dans expo_train les sinistres ? 

In [97]:
df_merged=pd.merge(expo_train, sin_train, on=['id', 'ANNEE'], how='left' )
df_merged

,Unnamed: 0,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,...,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE,NB,COUT
0,384538,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,...,0,1,0,0,0.0,0,5,2017,NaN,NaN
1,441079,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,...,0,1,0,0,0.0,0,9,2018,NaN,NaN
2,119668,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,...,0,1,0,0,0.0,0,11,2017,NaN,NaN
3,5124,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,...,0,1,0,0,0.0,0,13,2018,NaN,NaN
4,130065,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,...,0,1,0,0,0.0,0,14,2017,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155646,280004,0.680327,MEDIUM,PRINCIPALE,APPARTEMENT,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,...,0,1,0,0,0.0,0,499986,2017,NaN,NaN
155647,366975,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,9000.0,NIVEAU_1,...,0,1,1,0,0.0,0,499989,2016,NaN,NaN
155648,165418,0.569863,ALL_INCLUDE,PRINCIPALE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,...,0,1,0,0,NaN,0,499991,2018,NaN,NaN
155649,366439,0.597260,CONFORT,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,...,0,1,0,0,0.0,0,499993,2016,NaN,NaN


**ATTENTION!!** POurquoi est-ce qu'on ne retrouve aps id=5 dans les `sin_train` ? Dans `expo_train` il y a bien un sinistre type 1 en année 2 

In [99]:
list(df_merged)

['Unnamed: 0',
 'EXPO',
 'FORMULE',
 'TYPE_RESIDENCE',
 'TYPE_HABITATION',
 'NB_PIECES',
 'SITUATION_JURIDIQUE',
 'NIVEAU_JURIDIQUE',
 'VALEUR_DES_BIENS',
 'OBJETS_DE_VALEUR',
 'ZONIER',
 'NBSIN_TYPE1_AN1',
 'NBSIN_TYPE1_AN2',
 'NBSIN_TYPE1_AN3',
 'NBSIN_TYPE2_AN1',
 'NBSIN_TYPE2_AN2',
 'NBSIN_TYPE2_AN3',
 'id',
 'ANNEE',
 'NB',
 'COUT']

In [104]:
#On vire la colonne unnamed :0 : André => ancien numéro de ligne
df_merged= df_merged.drop('Unnamed: 0', axis=1)

In [106]:
df_merged[['NB', 'COUT']]=df_merged[['NB', 'COUT']].fillna(0)

In [107]:
df_merged

,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE,NB,COUT
0,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,0,1,0,0,0.0,0,5,2017,0.0,0.0
1,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,0,1,0,0,0.0,0,9,2018,0.0,0.0
2,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,0,1,0,0,0.0,0,11,2017,0.0,0.0
3,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,0,1,0,0,0.0,0,13,2018,0.0,0.0
4,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,0,1,0,0,0.0,0,14,2017,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155646,0.680327,MEDIUM,PRINCIPALE,APPARTEMENT,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A12,0,1,0,0,0.0,0,499986,2017,0.0,0.0
155647,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,9000.0,NIVEAU_1,B19,0,1,1,0,0.0,0,499989,2016,0.0,0.0
155648,0.569863,ALL_INCLUDE,PRINCIPALE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A9,0,1,0,0,NaN,0,499991,2018,0.0,0.0
155649,0.597260,CONFORT,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,C23,0,1,0,0,0.0,0,499993,2016,0.0,0.0


In [130]:
# Insurances companies are interested in modeling the Pure Premium, that is
# the expected total claim amount per unit of exposure for each policyholder
# in their portfolio:
df_merged["PurePremium"] = df_merged["COUT"] / df_merged["EXPO"]

# This can be indirectly approximated by a 2-step modeling: the product of the
# Frequency times the average claim amount per claim:
df_merged["Frequency"] = df_merged["NB"] / df_merged["EXPO"]
df_merged["AvgClaimAmount"] = df_merged["COUT"] / np.fmax(df_merged["NB"], 1)

In [131]:
df_merged

,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,...,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE,NB,COUT,PurePremium,Frequency,AvgClaimAmount
0,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,...,0,0.0,0,5,2017,0.0,0.0,0.0,0.0,0.0
1,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,...,0,0.0,0,9,2018,0.0,0.0,0.0,0.0,0.0
2,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,...,0,0.0,0,11,2017,0.0,0.0,0.0,0.0,0.0
3,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,...,0,0.0,0,13,2018,0.0,0.0,0.0,0.0,0.0
4,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,...,0,0.0,0,14,2017,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155646,0.680327,MEDIUM,PRINCIPALE,APPARTEMENT,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A12,...,0,0.0,0,499986,2017,0.0,0.0,0.0,0.0,0.0
155647,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,9000.0,NIVEAU_1,B19,...,0,0.0,0,499989,2016,0.0,0.0,0.0,0.0,0.0
155648,0.569863,ALL_INCLUDE,PRINCIPALE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A9,...,0,NaN,0,499991,2018,0.0,0.0,0.0,0.0,0.0
155649,0.597260,CONFORT,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,C23,...,0,0.0,0,499993,2016,0.0,0.0,0.0,0.0,0.0


In [132]:
df_merged.to_parquet('/mnt/data/interim/df_merged.gzip',compression='gzip')

In [ ]:
sin_train.descriv